In [13]:
import cv2
import numpy as np
from openvino.runtime import Core
from openvino_face_helpers.face_detector import FaceDetector
from openvino_face_helpers.utils import OutputTransform
import time
import os
import sys
import logging

# Get the current working directory and add the parent directory to the Python path
current_working_directory = os.getcwd()
sys.path.append(os.path.join(current_working_directory, ".."))

from helpers.helpers import (
    VideoPlayer,
)

In [14]:
logging.basicConfig(
    format="[ %(levelname)s ] - %(asctime)s - %(message)s", level=logging.DEBUG, stream=sys.stdout
    
)

In [15]:
class FaceDetectionPipeline:

    def __init__(self, fd_model_path):
        assert os.path.exists(fd_model_path), f"Model file not found at {fd_model_path}"
        self.core = Core()
        self.device = "CPU"
        # Specify the input size of detection model for reshaping. Example: 500 700. 
        # Pass (0, 0) for model default
        self.input_image_size = (0, 0)
        # detection threshold
        self.face_detection_confidence_threshold = 0.5
        # Scaling ratio for bboxes passed to face recognition.
        self.roi_scale_factor = 1.1
        # Specify the maximum output window resolution in (width x height) format. 
        # Example: 1280x720. Input frame size used by default.
        self.output_resolution = None
        self.face_detector = FaceDetector(self.core, 
                                            fd_model_path, 
                                            self.input_image_size,
                                            confidence_threshold=self.face_detection_confidence_threshold,
                                            roi_scale_factor=self.roi_scale_factor
                                            )
        
        self.face_detector.deploy(self.device)
        self.QUEUE_SIZE = 16
        
    def process(self, frame):
        orig_image = frame.copy()
        rois = self.face_detector.infer((frame,))
        
        if self.QUEUE_SIZE < len(rois):
            logging.warning('Too many faces for processing. Will be processed only {self.QUEUE_SIZE} of {len(rois)}')
            rois = rois[:self.QUEUE_SIZE]
        return rois

    def draw_detections(self, frame, detections, output_transform):
        size = frame.shape[:2]
        frame = output_transform.resize(frame)
        for roi in detections:
            
            xmin = max(int(roi.position[0]), 0)
            ymin = max(int(roi.position[1]), 0)
            xmax = min(int(roi.position[0] + roi.size[0]), size[1])
            ymax = min(int(roi.position[1] + roi.size[1]), size[0])
            xmin, ymin, xmax, ymax = output_transform.scale([xmin, ymin, xmax, ymax])
            
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 220, 0), 2)
            
            # Extract face region
            face_region = frame[ymin:ymax, xmin:xmax]

            # Convert face region to grayscale
            gray_face = cv2.cvtColor(face_region, cv2.COLOR_BGR2GRAY)

            # Apply Gaussian blur for smoother heatmap
            blurred_heatmap = cv2.GaussianBlur(gray_face, (5, 5), 0)

            # Apply colormap for visualization
            heatmap_colored = cv2.applyColorMap(blurred_heatmap, cv2.COLORMAP_JET)

            # Introduce random color shift for each frame
            color_shift = np.random.randint(0, 256, 3)
            heatmap_colored = heatmap_colored + color_shift

            # Ensure that the values stay within the valid range [0, 255]
            heatmap_colored = np.clip(heatmap_colored, 0, 255).astype(np.uint8)

            # Blend the heatmap with the original frame
            blended = cv2.addWeighted(face_region, 0.3, heatmap_colored, 0.7, 0)  # Adjust weights

            # Replace the original face region with the blended result
            frame[ymin:ymax, xmin:xmax] = blended


        return frame

    def run_async(self, source=0, required_fps=30, title="Face Detection"):
    
        frame_number = 0
        player = None
        
        try:
            # Create a video player
            player = VideoPlayer(source, fps=required_fps)
            # Start capturing
            start_time = time.time()
            player.start()

            while True:
                frame = player.next()
                
                if frame is None:
                    print("Source ended")
                    break
                

                if frame_number == 0:
                    output_transform = OutputTransform(frame.shape[:2], self.output_resolution)
                    if self.output_resolution:
                        self.output_resolution = output_transform.new_resolution
                    else:
                        self.output_resolution = (frame.shape[1], frame.shape[0])

                detections = self.process(frame)
                
                stop_time = time.time()
                total_time = stop_time - start_time
                frame_number += 1
                sync_fps = frame_number / total_time
                
                frame = self.draw_detections(frame, detections, output_transform)
                cv2.putText(frame, "Running Asynchronously", (5, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 2)
                cv2.putText(frame, f"{round(sync_fps, 2)} FPS", (5, 60), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 2)
                cv2.namedWindow(title, cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_AUTOSIZE)
                cv2.imshow(title, frame)
                key = cv2.waitKey(1)
                # escape = 27
                if key == 27:
                    break

        except KeyboardInterrupt:
            print("Processing interrupted by user.")
        finally:
            cv2.destroyAllWindows()
            if player is not None:
                player.stop()


    def run_sync(self, source=0):
        
        cap = cv2.VideoCapture(source)
        frame_number = 0
        start_time = time.time()
        
        try:
            while True:
                ret, frame = cap.read()
                
                if not ret:
                    print("Source ended or not found")
                    break
                
                if frame_number == 0:
                    output_transform = OutputTransform(frame.shape[:2], self.output_resolution)
                    if self.output_resolution:
                        self.output_resolution = output_transform.new_resolution
                    else:
                        self.output_resolution = (frame.shape[1], frame.shape[0])

                detections = self.process(frame)


                stop_time = time.time()
                total_time = stop_time - start_time
                frame_number += 1
                sync_fps = frame_number / total_time


                frame = self.draw_detections(frame, detections, output_transform)
                cv2.putText(frame, "Running Synchronously", (5, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 2)
                cv2.putText(frame, f"{round(sync_fps, 2)} FPS", (5, 60), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 2)
                cv2.imshow('Face recognition demo', frame)
                key = cv2.waitKey(1)
                # Quit
                if key in {ord('q'), ord('Q'), 27}:
                    break
        except KeyboardInterrupt:
            print("Processing interrupted by user.")
        finally:
            cv2.destroyAllWindows()
            cap.release()

In [16]:
def run_face_detection(source, model_path, required_fps, with_async=True):
    obj = FaceDetectionPipeline(model_path)
    if with_async:
        obj.run_async(source=source, required_fps=required_fps)
    else:
        obj.run_sync(source=source)

In [17]:
def main():
    source = 2
    fd_model_path = "/home/acer/workspace/intel_models/intel/face-detection-adas-0001/FP32/face-detection-adas-0001.xml"
    
    run_face_detection(source, fd_model_path, required_fps=60, with_async=True)

In [18]:
main()

[ INFO ] - 2023-12-23 19:29:44,228 - Reading Face Detection model /home/acer/workspace/intel_models/intel/face-detection-adas-0001/FP32/face-detection-adas-0001.xml


[ INFO ] - 2023-12-23 19:29:44,346 - The Face Detection model /home/acer/workspace/intel_models/intel/face-detection-adas-0001/FP32/face-detection-adas-0001.xml is loaded to CPU
